The first thing we do is importing several Python libraries that we need to do the exercise. There are:


1.   **Pandas** library (pd). It is used for working with data sets and has functions for analyzing, cleaning, exploring, and manipulating data. Pandas stands for Python Data Analysis
2.   **NumPy** library (np). It is used for working with arrays and has functions for working in domain of linear algebra, fourier transform, and matrices.
3.  **Matplotlib** libary. It is used to create graphs and plots by using Python scripts. It has a module named pyplot (plt) which makes things easy for plotting by providing feature to control line styles, font properties, formatting axes etc.
4. **Seaborn** library (sns). It is a Python data visualization library based on matplotlib and provides a high-level interface for drawing attractive and informative statistical graphics.
5. **Regular Expression** module (re). It is a sequence characters that forms a search pattern and can be used to check if a string contains the specified search pattern.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
sns.set()

In [ ]:
#import the datasets we will use # TO BE USED FOR EX 1 to 8
df_hotels = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2020/2020-02-11/hotels.csv')

# The dataset doesn’t have unique identifier column, so we add the ‘id’ column as a unique identifier. Here is the code we can use:
df_hotels = df_hotels.reset_index().rename(columns={'index':'id'})



#load the country dataset ### TO BE USED FOR EX 9 and 10
df_country = pd.read_csv('https://gist.githubusercontent.com/tadast/8827699/raw/f5cac3d42d16b78348610fc4ec301e9234f82821/countries_codes_and_coordinates.csv')
df_country['code'] = df_country['Alpha-3 code'].str.replace('"','').str.strip()




Now it is time for some exercises. Answer the following problems.

## EXERCISE 1

Create a function with one argument formed in DataFrame to check the data type, the number of null values, the percentage of null values and the number of unique values for each column.


## EXCERCISE 2
How many visitors are there who cancel the reservation and who don’t? And from that number draw conclusions about the proportions of each.

## EXCERCISE 3

*   For “City Hotel”, what is the percentage of canceled reservations?
*   For “Resort Hotel”, what is the percentage of canceled reservations?
*   What type of hotel that has the bigger percentage of canceled reservations?


## EXCERCISE 4
Filter data so that it only displays the visitors who don’t cancel the reservation and save the result in df_checkout variable.

## EXCERCISE 5

*   Show the number of reservations per month of arrival for each type of hotel!
*   Then in which month there are the most reservations in each type of hotel? Make a conclusion whether the trend is the same in both types of hotels?
*    Do as the previous point but with the name of the month that has been mapped into months in numbers.

note: for this and subsequent questions will use the ***df_checkout*** dataframe.

## EXCERCISE 6

*  Create a new column named arrival_date which contains complete information about the year, month, and date of arrival (example: 2022-12-27)
*  Change the column to datetime type


## EXCERCISE 7

Create a dataframe containing:
*   Daily reservation (df_daily_reservation)

## EXCERCISE 8

* What is the average ADR (Average Daily Rate) based on hotel type and customer type?
* Which type of customer has the highest the average of ADR in each type of hotel?

## EXCERCISE 9

By using the ‘df_country’ dataframe which contains the country name and country code information, show the 10 countries with the largest number of reservations. (P.S. you need to combine this dataframe with another right one)

## EXERCISE 10

* How many average guests stay for each reservation?
* Based on the dataset, what is the highest number of guests? Also show the reservation data row that has the highest number of guests.




# Answers